### Import Libraries

In [20]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud # for wordcloud
import matplotlib.pyplot as plt # for wordcloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, hamming_loss, f1_score, jaccard_score, accuracy_score
import pickle
from sklearn.multiclass import OneVsRestClassifier

## FEATURE EXTRACTION

### Import Data

In [21]:
# import a more balance train data
train_data = pd.read_csv("./Important_Data/7_first_600_manually_add_theme.csv") # obtain balance data from the first 600 data manually
test_data = pd.read_csv("./Important_Data/8_manually_create_balance_data.csv")

test_x = test_data["summary"]
test_y = test_data["theme"]

train_x = train_data["summary"]
train_y = train_data["theme"]

categories = ["corporate and business topics", 
              "labor and employment issues", 
              "privacy, security, and cyber matters", 
              "legal and crime stories", 
              "government actions and regulations", 
              "technology and digital trends", 
              "environment and climate topics", 
              "social issues and activism", 
              "healthcare and medicine", 
              "community and cultural events", 
              "international relations and trade", 
              "education and learning", 
              "consumer topics", 
              "infrastructure and development", 
              "energy and resources", 
              "political topics and protests", 
              "media and communication", 
              "financial policies and taxation", 
              "human rights and social justice", 
              "science, research, and innovation", 
              "disaster and crisis management", 
              "organized crime and trafficking", 
              "sports, entertainment, and leisure", 
              "other", 
              "military"]

y_encoded = []
for each_theme in train_y:
    each_row = []
    for category in categories:
        if category in each_theme.lower():
            each_row.append(1)
        else:
            each_row.append(0)
    y_encoded.append(each_row)

# convert to dataframe
y_encoded = pd.DataFrame(y_encoded, columns = categories)
train_y = y_encoded

y_encoded = []
for each_theme in test_y:
    each_row = []
    for category in categories:
        if category in each_theme.lower():
            each_row.append(1)
        else:
            each_row.append(0)
    y_encoded.append(each_row)

y_encoded = pd.DataFrame(y_encoded, columns = categories)
test_y = y_encoded   

### Word2Vec (FINAL)

In [22]:
# tokenize all the sentences
tokenized_sentences = [word_tokenize(each_line[0].lower()) for each_line in train_data["summary"]]

# train word2vec model
word2vec_model = Word2Vec(
    sentences = tokenized_sentences, 
    vector_size = 1000, 
    window = 5, 
    min_count = 1, 
    workers = 4
)

# generate document vectors
def vectorize_doc(each_line):
    # remove out of vocab words
    words = [word for word in each_line if word in word2vec_model.wv]
    return np.mean(word2vec_model.wv[words], axis = 0) if words else np.zeros(word2vec_model.vector_size)

# create feature vectors 
train_x = np.array([vectorize_doc(word_tokenize(each_line.lower())) for each_line in train_data["summary"]])
test_x = np.array([vectorize_doc(word_tokenize(each_line.lower())) for each_line in test_data["summary"]])

# save the model
with open("./Important_Data/word2vec_model.pkl", "wb") as f:
    pickle.dump(word2vec_model, f)
print("Model saved!")

Model saved!


### TFIDF

In [25]:
# word pattern
pattern = r"[a-zA-Z]+"

vectorizer = TfidfVectorizer(
    token_pattern = pattern, 
    stop_words = 'english', 
    ngram_range = (1, 2), # need to do the hyperparameter tuning for this later
    max_features = 100000 # can adjust to larger num too
)

vectorizer.fit(train_data["summary"])


# tokenization - create matrix
train_tokenized_features = vectorizer.transform(train_data["summary"])
test_tokenized_features = vectorizer.transform(test_data["summary"])


# create a dataframe
train_features = pd.DataFrame(
    data = train_tokenized_features.toarray(), 
    columns = vectorizer.get_feature_names_out()
)
test_features = pd.DataFrame(
    data = test_tokenized_features.toarray(), 
    columns = vectorizer.get_feature_names_out()
)


train_data["char_count"] = train_data["summary"].str.count(r"\S")
train_data["word_count"] = train_data["summary"].str.count(pattern)
train_data["avg_word_length"] = train_data["char_count"] / train_data["word_count"]
train_x = pd.concat([train_features, train_data.loc[:, "char_count": ]], axis = 1)

test_data["char_count"] = test_data["summary"].str.count(r"\S")
test_data["word_count"] = test_data["summary"].str.count(pattern)
test_data["avg_word_length"] = test_data["char_count"] / test_data["word_count"]
test_x = pd.concat([test_features, test_data.loc[:, "char_count": ]], axis = 1)


## MODEL SELECTIONS

### Random Forest Classifier (FINAL)

In [26]:
# label powerset
# it can capture label dependencies but if too many label combinations
# it will lead to poor performance

from skmultilearn.problem_transform import LabelPowerset

model = LabelPowerset(RandomForestClassifier(n_estimators = 100, 
                                             random_state = 59, 
                                             max_depth = 20, 
                                             min_samples_split = 2, 
                                             min_samples_leaf = 3))
model.fit(train_x, train_y)
pred = model.predict(test_x)

# save the model
with open("./Important_Data/random_forest_classifier.pkl", "wb") as f:
    pickle.dump(model, f)
print("Model saved!")


Model saved!


c:\Users\axlee\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [28]:
print(classification_report(
    test_y, 
    pred, 
    target_names = categories
))

print("Hamming Loss: ", hamming_loss(test_y, pred))
print("F1 Score: ", f1_score(
    test_y, 
    pred, 
    average = "macro"
))
print("Jaccard Score: ", jaccard_score(
    test_y, 
    pred, 
    average = "samples"
))
print("Accuracy: ", accuracy_score(test_y, pred))

                                      precision    recall  f1-score   support

       corporate and business topics       0.75      0.20      0.32        15
         labor and employment issues       1.00      0.40      0.57         5
privacy, security, and cyber matters       1.00      0.40      0.57         5
             legal and crime stories       0.21      0.62      0.31        16
  government actions and regulations       0.50      0.12      0.19        17
       technology and digital trends       1.00      0.22      0.36         9
      environment and climate topics       0.00      0.00      0.00         7
          social issues and activism       0.00      0.00      0.00         6
             healthcare and medicine       0.26      1.00      0.42         5
       community and cultural events       0.00      0.00      0.00         5
   international relations and trade       0.24      0.90      0.38        20
              education and learning       1.00      0.60      

c:\Users\axlee\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Softmax Regression (One vs All)

In [29]:
model = OneVsRestClassifier(LogisticRegression())
model.fit(train_x, train_y)

prediction = model.predict(test_x)
pred = (prediction > 0.001).astype(int)
pred_theme = [[categories[i] for i in range(len(categories)) if pred[j, i] == 1] for j in range(len(pred))]

c:\Users\axlee\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


KeyboardInterrupt: 

In [ ]:
print(classification_report(
    test_y, 
    pred, 
    target_names = categories
))

print("Hamming Loss: ", hamming_loss(test_y, pred))
print("F1 Score: ", f1_score(
    test_y, 
    pred, 
    average = "macro"
))
print("Jaccard Score: ", jaccard_score(
    test_y, 
    pred, 
    average = "samples"
))
print("Accuracy: ", accuracy_score(test_y, pred))

### Support Vector Machine (SVM)

In [ ]:
# label powerset
model = SVC(kernel = 'rbf', # can change to 'rbf'
            )
model = LabelPowerset(model)

model.fit(train_x, train_y)

pred = model.predict(test_x)
print("Accuracy: ", accuracy_score(test_y, pred))
print("Hamming Loss: ", hamming_loss(test_y, pred))

In [ ]:
print(classification_report(
    test_y, 
    pred, 
    target_names = categories
))

print("Hamming Loss: ", hamming_loss(test_y, pred))
print("F1 Score: ", f1_score(
    test_y, 
    pred, 
    average = "macro"
))
print("Jaccard Score: ", jaccard_score(
    test_y, 
    pred, 
    average = "samples"
))
print("Accuracy: ", accuracy_score(test_y, pred))